# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un dataset con reviews de películas.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [33]:
import pandas as pd

In [34]:
# Cargar el dataset de reseñas IMDB
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

# **Definir la función para buscar palabras**

In [35]:
def buscar_palabra(df, palabra):
    """
    Busca una palabra en las reviews del dataset y muestra
    en qué documentos aparece.
    """
    palabra = palabra.lower()  # Normaliza la palabra
    encontrados = []  # Guarda los índices donde se encuentra la palabra

    # Recorre todas las reseñas
    for i, review in enumerate(df['review']):
        if palabra in review.lower():  # Busca coincidencias
            encontrados.append(i)

    # Muestra resultados
    if encontrados:
        print(f"La palabra '{palabra}' aparece en {len(encontrados)} documentos:")
        for idx in encontrados[:10]:
            print(f"- Documento {idx}: {df.loc[idx, 'review'][:100]}...")
    else:
        print(f"No se encontró la palabra '{palabra}' en ninguna review.")


Aquí vamos a recibir el dataframe y la palabra que el usuario quiere buscar, convertimos todo a minúsculas con la función lower

Vamos a recorrer cada reseña y se verificamos si contiene la palabra buscada.
Si la contiene, se guarda el índice del review.

# Ejecución

In [36]:
# Pedir la palabra al usuario
palabra_usuario = input("Ingresa una palabra para buscar: ")

# Llamar a la función
buscar_palabra(df, palabra_usuario)


Ingresa una palabra para buscar:  wonderfull


La palabra 'wonderfull' aparece en 578 documentos:
- Documento 29: 'War movie' is a Hollywood genre that has been done and redone so many times that clichéd dialogue, ...
- Documento 72: I thought that Mukhsin has been wonderfully written. Its not just about entertainment. There's tonne...
- Documento 142: After hearing about George Orwell's prophetic masterpiece for all of my life, I'm now 37, but never ...
- Documento 499: Such a joyous world has been created for us in Pixar's A Bug's Life; we're immersed in a universe wh...
- Documento 572: I attended an advance screening of this film not sure of what to expect from Kevin Costner and Ashto...
- Documento 608: I have recently seen this production on DVD. It is the first time I have seen it since it was origin...
- Documento 712: Arguably the finest serial ever made(no argument here thus far) about Earthman Flash Gordon, Profess...
- Documento 908: Sometimes it's hard to be a pirate...............but by golly Miss Jean Peters has a lo

Solicitamos ingresar la palabra y luego se llama a la función con el dataset cargado.

## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

# Importar los datos

In [43]:
import pandas as pd

# Cargar dataset IMDB
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')


Cargamos las reseñas en un DataFrame df, igual que en la parte anterior.


# Construir el índice invertido

In [44]:
from collections import defaultdict

def construir_indice_invertido_rapido(df, columna='review'):
    """
    Construye un índice invertido usando defaultdict y set para velocidad.
    Cada palabra apunta a un set de índices de documentos donde aparece.
    """
    indice_invertido = defaultdict(set)

    for i, review in enumerate(df[columna]):
        if pd.isna(review):
            continue  # Ignorar valores nulos
        palabras = review.lower().split()
        for palabra in palabras:
            palabra = palabra.strip('.,!?"()[]')  # Limpiar signos de puntuación
            if palabra:
                indice_invertido[palabra].add(i)

    # Convertimos sets a listas
    for palabra in indice_invertido:
        indice_invertido[palabra] = list(indice_invertido[palabra])

    return indice_invertido

def buscar_en_indice(palabra, indice):
    """
    Busca una palabra en el índice invertido y devuelve
    la lista de documentos donde aparece.
    """
    return indice.get(palabra.lower(), [])


Aquí recorremos cada reseña y cada palabra.
Si la palabra no está en el diccionario, se crea; si ya está, se agrega el número del índice donde aparece.

# Construir el índice una vez

In [45]:
# Ejemplo con dataset pequeño primero para probar
df_pequeno = df.head(1000)  # puedes cambiar a df_large si quieres todo el dataset
indice_pequeno = construir_indice_invertido_rapido(df_pequeno)

print("Índice invertido rápido construido correctamente.")


Índice invertido rápido construido correctamente.


Esto genera el índice una sola vez, lo que evita búsquedas lentas en cada consulta.

# Uso del buscador

El usuario ingresa una palabra, y la búsqueda se realiza de forma instantánea utilizando el índice.

In [46]:
palabra = "great"  # Cambia por la palabra que quieras buscar
documentos = buscar_en_indice(palabra, indice_pequeno)

print(f"La palabra '{palabra}' aparece en {len(documentos)} documentos.")
print("Ejemplos de documentos:", documentos[:10])


La palabra 'great' aparece en 243 documentos.
Ejemplos de documentos: [1, 2, 513, 519, 528, 24, 25, 537, 539, 540]


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño.
      -  Corpus grande.
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

In [63]:
import pandas as pd
from collections import defaultdict
import time

# Cargar datasets
df_imdb = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df_rotten = pd.read_csv('/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv')

# Revisar columnas
print("Columnas IMDb:", df_imdb.columns)
print("Columnas Rotten Tomatoes:", df_rotten.columns)


Columnas IMDb: Index(['review', 'sentiment'], dtype='object')
Columnas Rotten Tomatoes: Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content'],
      dtype='object')


In [64]:
# Diccionarios de documentos
docs_imdb = df_imdb['review']
docs_rotten = df_rotten['review_content'].dropna()

def construir_indice_invertido(documentos):
    """
    Construye un índice invertido donde cada palabra apunta a un set de IDs de documentos.
    """
    indice = defaultdict(set)
    for doc_id, texto in documentos.items():
        for palabra in str(texto).lower().split():
            palabra = palabra.strip('.,!?"()[]')  # Limpiar signos de puntuación
            if palabra:
                indice[palabra].add(doc_id)
    return indice

# Crear índices
documentos_imdb = {i: texto for i, texto in enumerate(docs_imdb)}
documentos_rotten = {i: texto for i, texto in enumerate(docs_rotten)}

indice_imdb = construir_indice_invertido(documentos_imdb)
indice_rotten = construir_indice_invertido(documentos_rotten)

print("✅ Índices invertidos construidos para IMDb y Rotten Tomatoes.")


✅ Índices invertidos construidos para IMDb y Rotten Tomatoes.


In [65]:
def buscar_en_indice(indice, palabras):
    """
    Busca múltiples palabras en el índice invertido.
    Devuelve un set con todos los IDs de documentos que contienen alguna de las palabras.
    """
    resultados = set()
    for palabra in palabras:
        resultados |= indice.get(palabra.lower(), set())
    return resultados


In [66]:
def buscar_lineal(docs, palabras):
    """
    Busca palabras en los documentos de forma lineal usando pandas str.contains.
    """
    import re
    regex = "|".join([re.escape(p.lower()) for p in palabras])
    return docs[docs.str.lower().str.contains(regex, na=False)].index.tolist()


In [67]:
palabras_buscar = ["excellent", "boring", "love", "terrible"]

# IMDb
inicio = time.time()
buscar_en_indice(indice_imdb, palabras_buscar)
tiempo_indice_imdb = time.time() - inicio

inicio = time.time()
buscar_lineal(docs_imdb, palabras_buscar)
tiempo_lineal_imdb = time.time() - inicio

# Rotten Tomatoes
inicio = time.time()
buscar_en_indice(indice_rotten, palabras_buscar)
tiempo_indice_rotten = time.time() - inicio

inicio = time.time()
buscar_lineal(docs_rotten, palabras_buscar)
tiempo_lineal_rotten = time.time() - inicio


In [68]:
comparacion = pd.DataFrame({
    "Dataset": ["IMDb (50K reseñas)", "Rotten Tomatoes"],
    "Búsqueda lineal (s)": [tiempo_lineal_imdb, tiempo_lineal_rotten],
    "Búsqueda índice invertido (s)": [tiempo_indice_imdb, tiempo_indice_rotten]
})

print("\n⏱️ Comparación de tiempos IMDb vs Rotten Tomatoes\n")
display(comparacion)



⏱️ Comparación de tiempos IMDb vs Rotten Tomatoes



,Dataset,Búsqueda lineal (s),Búsqueda índice invertido (s)
0,IMDb (50K reseñas),1.039406,0.001465
1,Rotten Tomatoes,3.212717,0.003744
